In [1]:
from typing import *

import tensorflow as tf
import numpy as np
import pandas as pd
from tensorflow.python import keras
from tensorflow.python.keras import callbacks

import os
from datetime import datetime

from lib.dnn.utils import KerasTrainer
from core.utils.cached_optimizer import CachedOptimizer
from core.utils.file_storage import DropboxClient
from lib.dnn.layers import Delta, MovingAverage, Percentage


2022-02-26 18:28:29.922582: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-02-26 18:28:29.922698: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


Couldn't Connect to Postgres could not connect to server: Connection refused
	Is the server running on host "localhost" (::1) and accepting
	TCP/IP connections on port 5432?
could not connect to server: Connection refused
	Is the server running on host "localhost" (127.0.0.1) and accepting
	TCP/IP connections on port 5432?



In [10]:
FILES = [os.path.join("Data/Minutes", path) for path in os.listdir("Data/Minutes")[:2]]
DATA_PATH = os.path.abspath("temp/All.csv")
TRAINER_CONFIG = {
	"seq_len": 64,
	"hidden_layers": [1024, 2048, 1024, 64, 32],
	"delta": True,
	"percentage": True,
	"hidden_activation": "relu",
	"optimizer": "adam",
	"loss": "binary_crossentropy",
	"average_window": 0
}


In [3]:
class ModelCheckpointCallback(callbacks.Callback):
	
	def __init(self, *args, **kwargs):
		super(ModelCheckpointCallback, self).__init(*args, **kwargs)
		self.__dropbox_client = DropboxClient()
	
	def __save_model(selIt is. And it f, model: keras.Model) -> str:
		file_name = f"{datetime.now().timestamp()}.h5"
		model.save(file_name)
		return file_name
		
	def __upload_model(self, model_path: str):
		self.__dropbox_client.upload_file(model_path)
	
	def on_epoch_end(self, epoch, logs=None):
		print("[+]Saving Model...")
		path = self.__save_model(self.model)
		print("[+]Uploading Model...")
		self.__upload_model(path)


In [4]:
class Trainer(KerasTrainer):
	def __init__(self, seq_len, hidden_layers: List[int], *args,
					loss="binary_crossentropy",
					optimizer="adam",
					hidden_activation="relu",
					delta=True,
					average_window=None,
					percentage=True,
					**kwargs):
		super(Trainer, self).__init__(*args, **kwargs)
		self.__seq_len = seq_len
		self.__hidden_layers = hidden_layers
		self.__loss = loss
		self.__optimizer = optimizer
		self.__hidden_activation = hidden_activation
		self.__delta = delta
		self.__percentage = percentage
		self.__average_window = average_window
		if average_window is not None:
			self.__average_window = int(self.__seq_len * average_window)

	def __get_currency_pairs(self, df: pd.DataFrame) -> List[Tuple[str, str]]:
		DELIMITER = "/"
		return [
			(pair.split(DELIMITER)[0], pair.split(DELIMITER)[1]) for pair in
			set(df["base_currency"] + DELIMITER + df["quote_currency"])
		]  # TODO FIND A CLEAR WAY

	def __prepare_for_pair(self, sequence: np.ndarray, seq_len: int):
		data_len = sequence.shape[0] - seq_len
		X = np.zeros((data_len, seq_len))
		y = np.zeros((data_len,))
		for i in range(data_len):
			X[i] = sequence[i:i + seq_len]
			if sequence[i] > sequence[i - 1]:
				y[i] = 1
			else:
				y[i] = 0
		return X, y

	def _prepare_data(self, data: pd.DataFrame) -> Tuple[np.ndarray, np.ndarray]:
		currency_pairs = self.__get_currency_pairs(data)

		X = []
		y = []

		for base_currency, quote_currency in currency_pairs:
			pair_sequence = data[data["base_currency"] == base_currency][
				data[data["base_currency"] == base_currency]["quote_currency"] == quote_currency][
				"c"].to_numpy()  # TODO
			pair_X, pair_y = self.__prepare_for_pair(pair_sequence, self.__seq_len)
			X += list(pair_X)
			y += list(pair_y)

		return np.array(X), np.array(y)

	def _create_model(self) -> keras.Model:
		input_layer = keras.layers.Input(shape=self.__seq_len)

		pre_hidden_layer = input_layer

		if self.__average_window != 0:
			pre_hidden_layer = MovingAverage(self.__average_window)(pre_hidden_layer)

		pre_hidden_layer_copy = pre_hidden_layer[:, : -1]

		if self.__delta:
			pre_hidden_layer = Delta()(pre_hidden_layer)
		else:
			pre_hidden_layer = pre_hidden_layer[:, 1:]

		if self.__percentage:
			pre_hidden_layer = tf.concat(
				(
					tf.reshape(pre_hidden_layer, (-1, 1, pre_hidden_layer.shape[1])),
					tf.reshape(pre_hidden_layer_copy, (-1, 1, pre_hidden_layer.shape[1]))
				),
				axis=1
			)
			pre_hidden_layer = Percentage()(pre_hidden_layer)

		output_layer = keras.layers.Dense(1, activation="sigmoid")

		if len(self.__hidden_layers) == 0:
			output_layer = output_layer(pre_hidden_layer)
		else:
			hidden_layer = keras.layers.Dense(self.__hidden_layers[0], activation=self.__hidden_activation)(
				pre_hidden_layer)
			for layer_size in self.__hidden_layers[1:]:
				hidden_layer = keras.layers.Dense(layer_size, activation=self.__hidden_activation)(hidden_layer)
			output_layer = output_layer(hidden_layer)

		model = keras.Model(inputs=input_layer, outputs=output_layer)

		return model

	def _compile_model(self, model: keras.Model):
		model.compile(loss=self.__loss, optimizer=self.__optimizer, metrics=["accuracy"])


In [5]:
def merge_data(files=None, export_path=DATA_PATH):
	if files is None:
		files = FILES
	df = None
	for i, path in enumerate(files):
		pair_df = pd.read_csv(path, index_col=0)
		if df is None:
			df = pair_df
			continue
		df = df.append(pair_df)
		print(f"{(100 * (i+1))/len(files): .2f} %", end="\r")
	df.to_csv(export_path)
	del df


In [11]:
merge_data()


In [12]:
trainer = Trainer(
	**TRAINER_CONFIG, 
	export_path="finalModel.h5", 
	data_path=DATA_PATH, 
	epochs=50, 
	fit_params={
		"callbacks": [ModelCheckpointCallback()]
	})
trainer.start()


[+]Starting Training...
[+]Loading Data: /home/abreham/Projects/Personal_Projects/Mini_Projects/ForexTrader/RTrader/RTrader/temp/All.csv
